In [ ]:
import pandas as pd

# Load and inspect the uploaded file to understand its structure and sample data
file_path = r"Formations_Cours.csv"
courses_df = pd.read_csv(file_path)
courses_df.head()         

,ID_FORMATION,DOMAINE_CATGEGORIE,SOUS_DOMAINE_CATEGORIE,FORMATION_COURS
0,1,Achats,Management et métiers des achats,Responsable Achats
1,2,Achats,Management et métiers des achats,Management opérationnel des achats
2,3,Achats,Management et métiers des achats,Formation au métier d'acheteur
3,4,Achats,Management et métiers des achats,Formation pratique à l'achat
4,5,Achats,Management et métiers des achats,Formation assistant achat


Upload excel file of city and fromation:

In [ ]:
city_file_path = r"City.csv"
cities_df = pd.read_csv(city_file_path , encoding='ISO-8859-1')
cities_df.head()

,Nom
0,Aix-en-Provence
1,Amiens
2,Angers
3,Annecy
4,Argenteuil


Google Map searching and webscraping 

In [ ]:
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium import webdriver
import pandas as pd
import time
import pandas as pd
     

path = '/Users/yhema/Downloads/chromedriver-win64/chromedriver-win64/chromedriver.exe'
service = Service(executable_path=path)
driver = webdriver.Chrome(service=service)

results = []
cookies_accepted = False  # Flag to check if cookies were accepted

for course in courses_df['FORMATION_COURS']:    
    formation = f'https://www.google.fr/maps/search/Formation+{course.replace(" ", "+")}/'

    driver.get(formation)

    # Accept cookies only once
    if not cookies_accepted:
        accept_button_xpaths = [
            '//button[contains(text(),"Accept")]',
            '//button[contains(text(),"I agree")]',
            '//button[contains(text(),"Tout accepter")]',  # for French
            '//button[contains(@aria-label, "Tout accepter")]'
        ]
        
        # Try to locate and click the "Accept" cookies button
        for xpath in accept_button_xpaths:
            try:
                accept_button = WebDriverWait(driver, 10).until(
                    EC.element_to_be_clickable((By.XPATH, xpath))
                )
                accept_button.click()
                print("Cookies accepted.")
                cookies_accepted = True  # Set flag to True after clicking
                break
            except Exception as e:
                pass  # If no button found, continue
    
    # Extract elements containing the company names from aria-label attribute
    companies = driver.find_elements(By.XPATH, '//a[contains(@class,"hfpxzc")]')
    
    for company in companies:
        company_name = company.get_attribute("aria-label")
        
 
        try:
            # Click on the company to reveal more details
            try:
                company.click()
                # Small delay to allow the content to load
                time.sleep(1)
            except Exception:
                print("Click on company element failed.")

            # Wait for the address element with `Io6YTe` class to load
            WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.CLASS_NAME, "Io6YTe"))
            )
            
            # Extract the address after clicking
            address_element = driver.find_element(By.CLASS_NAME, "Io6YTe")
            address = address_element.text
        except Exception:
            address = "N/A" 
            
            
        try:
            additional_info = company.find_element(By.XPATH, './/ancestor::div[contains(@class,"Nv2PK")]//span[contains(@class,"UsdlK")]')
            additional_info_text = additional_info.text
        except Exception:
            additional_info_text = "N/A"  # In case UsdlK info is not found
            
        try:
            website_link = company.find_element(By.XPATH, './/ancestor::div[contains(@class,"Nv2PK")]//a[contains(@class,"S9kvJb")]').get_attribute("href")
        except Exception:
            website_link = "N/A"

        # Append data to results
        results.append({
            'Course': course,
            'Company Name': company_name,
            'Address': address,
            'Additional Info': additional_info_text,
            'Website': website_link
        })

# Quit the driver after scraping
driver.quit()

# Convert results to DataFrame and save as CSV
results_df = pd.DataFrame(results)
results_df.to_csv('output2_results.csv', index=False)


Cookies accepted.
